## Se necessário instalar as bibliotecas adicionais: 

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

## Importação das bibliotecas e instanciar as variáveis

In [1]:
#Requests do HTTP
import requests

#Criação e manipulação do dataframe
import pandas as pd

#Mineração de dados
from bs4 import BeautifulSoup

#Criação de valores nulos
import numpy as np

#Pra ver o tempo de run do crawler 
from time import time

#Lista de URL's de resoluções de janeiro até fevereiro de 2022
lista_urls = ['https://www.in.gov.br/web/dou/-/resolucao-re-n-175-de-20-de-janeiro-de-2022-375565424',
             'https://www.in.gov.br/web/dou/-/resolucao-re-n-255-de-27-de-janeiro-de-2022-376898876',
             'https://www.in.gov.br/web/dou/-/resolucao-re-n-361-de-3-de-fevereiro-de-2022-378387845',
             'https://www.in.gov.br/web/dou/-/resolucao-re-n-430-de-10-de-fevereiro-de-2022-379899450',
             'https://www.in.gov.br/web/dou/-/resolucao-re-n-522-de-17-de-fevereiro-de-2022-381447439']

## Desenvolvimento das funções

In [2]:
def preencher_nulos(dataframe):
    
    '''Pega as 3 primeiras colunas do dataframe (resoluções, empresas e autorizações) e utiliza o método ffill
    para preencher os valores nulos'''
    
    colunas = dataframe.columns[:3].tolist()
    dataframe[colunas] = dataframe[colunas].ffill()
    return dataframe

def minerar_pagina(url):
    
    '''Função para entrar na url e minerar todas as tags <p> do site e adicionando elas em uma lista para
    então depois fazer o processamento de texto dentro dessa lista e adicionar as respectivas informações
    nas colunas certas no formato de dataframe do Pandas'''
    
    #Instanciando o tempo inicial 
    tempo_inicial = time()
    
    #Fazendo o request da página e passando para o BeautifulSoup    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Juntar todos os parágrafos em uma lista 
    info = [marca.text for marca in soup.find_all('p', class_='dou-paragraph')]

    #É possível pegar as resoluções porque possuem classe única no HTML
    lista_resolucoes = []
    lista_resolucoes.append(soup.find('p', class_='identifica').text)
    
    #Todas as colunas que não possuem dados faltando e não precisam de tratamento adicional
    lista_marcas = [produto.split(':')[1][1:] for produto in info if 'NOME DO PRODUTO E MARCA' in produto]
    lista_processos = [processo.split(':')[1][1:] for processo in info if 'NUMERO DE PROCESSO' in processo]
    lista_registros = [registro.split(':')[1][1:] for registro in info if 'NUMERO DE REGISTRO' in registro]
    lista_vendas = [venda.split(':')[1][1:] for venda in info if 'VENDA E EMPREGO' in venda]
    lista_apresentacoes = [apresentacao.split(':')[1][1:] for apresentacao in info if 'APRESENTAÇÃO' in apresentacao]
    lista_validades = [validade.split(':')[1][1:] for validade in info if 'VALIDADE DO PRODUTO' in validade]
    lista_categorias = [categoria.split(':')[1][1:] for categoria in info if 'CATEGORIA' in categoria]
    lista_assuntos = [assunto.split(':')[1][1:] for assunto in info if 'ASSUNTO DA PETIÇÃO' in assunto]
    
    #Como é necessário que as colunas tenham o mesmo número de linhas, e só possui uma resolução por página, é
    #necessário preencher o resto das linhas da lista de resoluções com valores nulos para depois repetir eles
    for i in range(len(lista_marcas) - 1):
        lista_resolucoes.append(np.nan)
        
    #Instanciar as listas vazias
    empresas, autorizacoes, vencimentos, expedientes = ([] for i in range(4))

    #Aqui são as colunas que não possuem o mesmo número de linhas e precisam de tratamento
    #Para criar o mesmo número de linhas para todas, se um valor não está na linha que deveria estar,
    #será criado um valor nulo no lugar dele
    contador = 0  #Para verificar em qual linha estou e qual será a próxima
    for linha in info:
        #Sempre que tiver uma empresa, terá que ter uma autorização, então a lógica para ambos é a mesma
        if 'NOME DA EMPRESA' in linha:
            empresas.append(linha.split(':')[1][1:])
        
        if 'AUTORIZAÇÃO' in linha:
            autorizacoes.append(linha.split(':')[1][1:])
            
        if 'VENCIMENTO' in linha:
            vencimentos.append(linha.split(':')[1][1:])

        if 'PRODUTO E MARCA' in linha:
            if 'AUTORIZAÇÃO' not in ultima_linha:
                empresas.append(np.nan)
                autorizacoes.append(np.nan)

        if 'APRESENTAÇÃO' in linha:
            if 'VENCIMENTO' not in ultima_linha:
                vencimentos.append(np.nan)
                
        #A coluna expedientes segue a lógica de verificar a próxima linha, pois não é possível verificar a última
        if 'ASSUNTO DA PETIÇÃO' in linha:
            if contador == (len(info) - 1):
                expedientes.append(np.nan)
                break
                
            if 'EXPEDIENTE' in info[contador + 1]:
                expedientes.append(info[contador + 1].split(':')[1][1:])
                
            else:
                expedientes.append(np.nan)
                
        contador += 1
                
        #Crio a variável de última linha, e como é a última coisa dentro do for, no próximo loop ela será a linha anterior
        ultima_linha = linha 
    
    #Dicionário que vai ser passado para a criação do dataframe local
    dicionario = {'RESOLUCAO': [resolucao for resolucao in lista_resolucoes],
                  'EMPRESA': empresas,
                  'AUTORIZACAO': autorizacoes,
                  'MARCA': [marca for marca in lista_marcas],
                 'PROCESSO': [processo for processo in lista_processos],
                 'REGISTRO': [registro for registro in lista_registros],
                 'VENDA E EMPREGO': [venda for venda in lista_vendas],
                  'VENCIMENTO': vencimentos,
                 'APRESENTACAO': [apresentacao for apresentacao in lista_apresentacoes],
                 'VALIDADE PRODUTO': [validade for validade in lista_validades],
                 'CATEGORIA': [categoria for categoria in lista_categorias],
                 'ASSUNTO PETICAO': [assunto for assunto in lista_assuntos],
                 'EXPEDIENT E PETICAO': expedientes}
    
    #Adiciono o dataframe que foi criado da url a lista de todos os dataframes gerados
    lista_dataframes.append(pd.DataFrame(data=dicionario))
    
    #Tempo que demorou para minerar uma página inteira
    print(f'Essa página demorou {time() - tempo_inicial} segundos.')
    
#Juntar todos os dataframes gerados de cada página
def juntar_dataframes(lista_de_dataframes):
    dataframe = pd.concat(lista_de_dataframes)
    return dataframe

#Criar a coluna vazia de versão dentro do dataframe
def criar_coluna_versao(dataframe):
    dataframe['VERSAO'] = ""
    
def verificar_tamanho(dataframe):
    return(f'O dataframe criado possui {dataframe.shape[0]} linhas e {dataframe.shape[1]} colunas.')
    
### Daqui para baixo são as funções de tratamento de dados, não são necessárias para o pedido oficial do cliente
    
#Padronizar todas as linhas das colunas de objetos em letras maiúsculas, importante para o tratamento de dados
def transformar_maiusculo(dataframe):
    colunas = dataframe.select_dtypes(include='object')
    for coluna in colunas:
        dataframe[coluna] = dataframe[coluna].str.upper()
    
#Separar a coluna categoria em ID e categoria, depois transformar a coluna ID para int    
def separar_categoria(dataframe):
    dataframe[['ID_CATEGORIA', 'CLASSE_CATEGORIA']] = dataframe.CATEGORIA.str.split(' ', 1, expand=True)
    dataframe['ID_CATEGORIA'] = dataframe['ID_CATEGORIA'].astype(int)
    dataframe.drop(columns=['CATEGORIA'], inplace=True)


## Minerando página por página

In [3]:
#Lista vazia para dar o concat nos dataframes
lista_dataframes = []

#Iterando url por url e minerando a página
for url in lista_urls:
    minerar_pagina(url)

#Criar o dataframe pedido pelo cliente e exportar ele para ter um backup antes de realizar o tratamento
df = juntar_dataframes(lista_dataframes)

#Funções que precisam estar na planilha de backup
preencher_nulos(df)
criar_coluna_versao(df)

#Verificar a quantidade de informações mineradas
print(f'BACKUP: {verificar_tamanho(df)}')

#Salvando a planilha de backup
df.to_excel('Planilha-Anvisa-Backup.xlsx', index=False)

#Funções de tratamento de dados
transformar_maiusculo(df)
separar_categoria(df)

#Verificar a quantidade de informações mineradas após tratamento
print(f'TRATAMENTO: {verificar_tamanho(df)}')

#Salvando a planilha tratada
df.to_excel('Planilha-Anvisa-Tratada.xlsx', index=False)

Essa página demorou 1.7073822021484375 segundos.
Essa página demorou 0.5351197719573975 segundos.
Essa página demorou 0.6211390495300293 segundos.
Essa página demorou 0.5201160907745361 segundos.
Essa página demorou 0.5411214828491211 segundos.
BACKUP: O dataframe criado possui 991 linhas e 14 colunas.
TRATAMENTO: O dataframe criado possui 991 linhas e 15 colunas.
